In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'mapie',
    'causal',
    'wkmeans',
]
configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='halving',
        train_start=datetime(2018,1,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='random', search_type='wkmeans', search_subtype='wasserstein',
        n_trials=500, n_models=10, n_jobs=1, debug=False,
    ),
]
# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")